In [5]:
import os
import time
import openpyxl
from openai import OpenAI
from pathlib import Path

# OpenAI API 客户端（阿里云千问）
client = OpenAI(
    api_key="sk-038c3b8e36ce43e4a94156257a766bc4",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

print(os.getcwd())


c:\Users\Nack\Desktop\Nack-graduation-project\LLM\Qwen


In [ ]:
IMAGE_FOLDER = "images"  # 请修改为你的图片文件夹路径
OUTPUT_EXCEL = "images/image_descriptions.xlsx"  # 结果保存的Excel文件

# 支持的图片格式
VALID_EXTENSIONS = {".jpg", ".png", ".jpeg"}

# 创建 Excel 工作簿和工作表
wb = openpyxl.Workbook()
ws = wb.active
ws.append(["图片路径", "图片描述"])  # 添加表头


In [ ]:
import base64

def describe_image(image_path, client):
    """调用 API 获取图片描述"""
    try:
        # 读取图片并进行 Base64 编码
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")

        # 发送请求
        response = client.chat.completions.create(
            model="qwen2.5-vl-72b-instruct",
            messages=[
                {"role": "user", "content": [
                    # {"type": "text", "text": "你是一个机械制造领域的大师，现在有一个待安装SMA的盒子，共有20空位。空位中已有2个安装完成SMA，请你为剩下的空位规划安装顺序，使得安装效率最高。告诉我安装顺序坐标。"},
                    {"type": "text", "text": "你是一位机械制造领域的专家，擅长优化安装流程以提升效率。我将提供一张图片，图片中展示了一个待安装SMA（表面贴装组件）的盒子，盒子共有20个空位，其中2个空位已完成安装。请你根据图片识别盒子的空位分布及已完成安装的位置，并规划剩余空位的最优安装顺序，以最小化工具移动距离。输出时，请列出每个空位的安装顺序及其对应坐标，并说明优化逻辑。如果图片信息不清晰或存在歧义，请提出具体问题以便进一步确认。"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]}
            ]
        )

        # 获取返回结果
        print(f"response: {response}")
        return response.choices[0].message.content
    except Exception as e:
        print(f"处理 {image_path} 时出错: {e}")
        return "API 调用失败"


In [8]:
# 遍历文件夹中的所有图片
for image_file in os.listdir(IMAGE_FOLDER):
    image_path = os.path.join(IMAGE_FOLDER, image_file)
    image_path = os.path.abspath(image_path)  

    # 检查是否是图片文件
    if Path(image_path).suffix.lower() in VALID_EXTENSIONS:
        print(f"处理图片: {image_path}")
        description = describe_image(image_path, client)
        
        # 存入 Excel
        ws.append([image_path, description])
        
        # 适当延迟，避免 API 速率限制
        time.sleep(1)

# 保存 Excel 文件
wb.save(OUTPUT_EXCEL)
print(f"所有图片处理完成，结果已保存至 {OUTPUT_EXCEL}")

处理图片: c:\Users\Nack\Desktop\Nack-graduation-project\LLM\Qwen\images\图1.jpg
response: ChatCompletion(id='chatcmpl-5eac3bf3-bc93-90d2-a478-41d5182932af', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='为了提高安装效率，我们可以采用一种策略，即从已安装SMA的两个位置开始，逐步向四周扩展。这样可以减少机械臂或操作人员的移动距离，从而提高安装速度。根据图片中的布局，我们假设左上角为坐标(1,1)，右下角为坐标(5,4)。\n\n已安装SMA的位置为：\n- (1,1)\n- (3,2)\n\n接下来的安装顺序可以按照以下步骤进行：\n\n1. 从(3,2)开始，向右和向下扩展：\n   - (4,2)\n   - (5,2)\n   - (3,3)\n   - (4,3)\n   - (5,3)\n   - (3,4)\n   - (4,4)\n   - (5,4)\n\n2. 然后从(1,1)开始，向右和向下扩展：\n   - (2,1)\n   - (3,1)\n   - (4,1)\n   - (5,1)\n   - (1,2)\n   - (2,2)\n   - (1,3)\n   - (2,3)\n   - (1,4)\n   - (2,4)\n\n最终的安装顺序坐标为：\n1. (4,2)\n2. (5,2)\n3. (3,3)\n4. (4,3)\n5. (5,3)\n6. (3,4)\n7. (4,4)\n8. (5,4)\n9. (2,1)\n10. (3,1)\n11. (4,1)\n12. (5,1)\n13. (1,2)\n14. (2,2)\n15. (1,3)\n16. (2,3)\n17. (1,4)\n18. (2,4)\n\n这样可以确保安装路径尽量短，提高整体安装效率。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None,